# Eurostat

In [1]:
from eurostat import germany as ger

In [2]:
# Population_by_age_groups_and_sex
population_by_age_groups_and_sex = ger.Population_by_age_groups_and_sex()

In [ ]:
# Population_structure
population_structure = ger.Population_structure()

In [ ]:
# Population_by_citizenship
population_by_citizenship = ger.Population_by_citizenship()

In [ ]:
# Fertility_and_mortality
fertility_and_mortality = ger.Fertility_and_mortality()

In [ ]:
# Living_conditions
living_conditions = ger.Living_conditions()

In [ ]:
# Education
education = ger.Education()

In [ ]:
# Culture_and_tourism
culture_and_tourism = ger.Culture_and_tourism()

In [ ]:
# Labour_market
labour_market = ger.Labour_market()

In [ ]:
# Transport
transport = ger.Transport()

In [ ]:
# Environment
environment = ger.Environment()